<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n50_x4_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n50_x4_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...          y_hat         e_hat
rep  index                        ...                             
0.0  0.0    30.297779  47.185462  ...   88183.236572  8.565830e+05
     1.0    42.978980  16.321746  ...   10158.819646 -3.937125e+06
     2.0    14.329744  38.722955  ...    3076.637952  2.908659e+04
     3.0    44.627495  41.280672  ...  113665.929741 -2.407224e+06
     4.0    11.813375  47.298919  ...    2016.565459  3.624571e+04
...               ...        ...  ...            ...           ...
99.0 45.0   35.766155  21.597023  ...   36533.786892 -5.310072e+05
     46.0   32.199017  28.113223  ...    8491.830677  6.625117e+05
     47.0   13.429367  22.819386  ...  -31954.056517 -2.979270e+04
     48.0   44.142088  47.780448  ...  127013.223494 -3.051099e+06
     49.0   16.340171  25.314628  ...  -25426.763325 -6.786924e+04

[5000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...          y_hat         e_hat
rep  index                        ...                             
0.0  0.0    30.297779  47.185462  ...   88183.236572  8.565830e+05
     1.0    42.978980  16.321746  ...   10158.819646 -3.937125e+06
     2.0    14.329744  38.722955  ...    3076.637952  2.908659e+04
     3.0    44.627495  41.280672  ...  113665.929741 -2.407224e+06
     4.0    11.813375  47.298919  ...    2016.565459  3.624571e+04
...               ...        ...  ...            ...           ...
99.0 45.0   35.766155  21.597023  ...   36533.786892 -5.310072e+05
     46.0   32.199017  28.113223  ...    8491.830677  6.625117e+05
     47.0   13.429367  22.819386  ...  -31954.056517 -2.979270e+04
     48.0   44.142088  47.780448  ...  127013.223494 -3.051099e+06
     49.0   16.340171  25.314628  ...  -25426.763325 -6.786924e+04

[5000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,30.297779,47.185462,41.121063,9.443700e+05,9.447662e+05,88183.236572,8.565830e+05
1,0.0,1.0,42.978980,16.321746,13.148890,-3.927430e+06,-3.926966e+06,10158.819646,-3.937125e+06
2,0.0,2.0,14.329744,38.722955,29.909199,3.194629e+04,3.216322e+04,3076.637952,2.908659e+04
3,0.0,3.0,44.627495,41.280672,38.714361,-2.294090e+06,-2.293558e+06,113665.929741,-2.407224e+06
4,0.0,4.0,11.813375,47.298919,23.497441,3.806834e+04,3.826227e+04,2016.565459,3.624571e+04
...,...,...,...,...,...,...,...,...,...
4995,99.0,45.0,35.766155,21.597023,33.325936,-4.948910e+05,-4.944734e+05,36533.786892,-5.310072e+05
4996,99.0,46.0,32.199017,28.113223,16.117397,6.706323e+05,6.710036e+05,8491.830677,6.625117e+05
4997,99.0,47.0,13.429367,22.819386,30.436637,-6.193931e+04,-6.174676e+04,-31954.056517,-2.979270e+04
4998,99.0,48.0,44.142088,47.780448,39.351072,-2.924619e+06,-2.924085e+06,127013.223494,-3.051099e+06


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...          y_hat         e_hat
rep  index                        ...                             
0.0  0.0    30.297779  47.185462  ...   88183.236572  8.565830e+05
     1.0    42.978980  16.321746  ...   10158.819646 -3.937125e+06
     2.0    14.329744  38.722955  ...    3076.637952  2.908659e+04
     3.0    44.627495  41.280672  ...  113665.929741 -2.407224e+06
     4.0    11.813375  47.298919  ...    2016.565459  3.624571e+04
...               ...        ...  ...            ...           ...
99.0 45.0   35.766155  21.597023  ...   36533.786892 -5.310072e+05
     46.0   32.199017  28.113223  ...    8491.830677  6.625117e+05
     47.0   13.429367  22.819386  ...  -31954.056517 -2.979270e+04
     48.0   44.142088  47.780448  ...  127013.223494 -3.051099e+06
     49.0   16.340171  25.314628  ...  -25426.763325 -6.786924e+04

[5000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
3.0,23.0,21.0,7269261192387.1875,168316516756600.5,330420963290.3267,8415825837830.025,25.359752020283594,5.679234860167526e-11,0.9999999999716038,Reject005=0 : Heteroscedasticity
5.0,21.0,23.0,31758093341585.926,175133648153199.84,1587904667079.2964,7960620370599.993,5.035082980278426,0.0004251799558763203,0.9997874100220618,Reject005=0 : Heteroscedasticity
6.0,20.0,24.0,292348746510.0689,293289590911869.0,15386776132.10889,12751721343994.305,836.0151885177991,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
12.0,27.0,17.0,3392093363209.704,244487188032175.84,130465129354.21939,15280449252010.99,114.47302282530772,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
14.0,23.0,21.0,1710618044391.6714,106528374764963.95,77755365654.16689,5326418738248.197,68.205719359099,1.1102230246251565e-15,0.9999999999999994,Reject005=0 : Heteroscedasticity
15.0,23.0,21.0,104446781655493.36,113100099479617.22,4747580984340.607,5655004973980.861,1.1859775434141395,0.6877830923261088,0.6561084538369456,Reject005=1 : Homoscedasticity
18.0,24.0,20.0,123853441351750.39,117178130077283.66,5384932232684.8,6167270004067.561,1.1353237710479906,0.7591632579002576,0.6204183710498712,Reject005=1 : Homoscedasticity
19.0,20.0,24.0,6106257393793.66,363458390262944.6,321381968094.40314,15802538707084.549,49.60190380995624,9.057199434892027e-13,0.9999999999995471,Reject005=0 : Heteroscedasticity
20.0,16.0,28.0,5296290006523.389,212894247222474.66,353086000434.8926,7884972119350.913,22.969636369205602,3.695873740383604e-08,0.9999999815206313,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,22.0,22.0,132474132630294.17,21779538638561.84,6308292030014.008,1037120887550.5638,0.16440597274445787,7.900063770715098e-05,Reject005=0 : Heteroscedasticity
1.0,22.0,22.0,77359260726095.17,59207854269872.55,3683774320290.246,2819421631898.693,0.7653622037510021,0.5358307704612592,Reject005=1 : Homoscedasticity
2.0,23.0,21.0,184367096361677.16,49236418242009.06,8380322561894.416,2461820912100.453,0.29249037380258525,0.006221022231016117,Reject005=0 : Heteroscedasticity
3.0,23.0,21.0,7269261192387.1875,168316516756600.5,330420963290.3267,8415825837830.025,25.359752020283594,5.679234860167526e-11,Reject005=0 : Heteroscedasticity
4.0,19.0,25.0,195709473138661.6,124727911136657.58,10872748507703.422,5196996297360.732,0.48435679144002425,0.09022332985953098,Reject005=1 : Homoscedasticity
5.0,21.0,23.0,31758093341585.926,175133648153199.84,1587904667079.2964,7960620370599.993,5.035082980278426,0.0004251799558763203,Reject005=0 : Heteroscedasticity
6.0,20.0,24.0,292348746510.0689,293289590911869.0,15386776132.10889,12751721343994.305,836.0151885177991,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
7.0,20.0,24.0,121981762619097.03,86162769913710.95,6420092769426.159,3746207387552.65,0.5886315029373471,0.21517168178438026,Reject005=1 : Homoscedasticity
8.0,25.0,19.0,125276379136549.39,44889809216300.64,5219849130689.558,2493878289794.48,0.47148184557697254,0.09682868801108518,Reject005=1 : Homoscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    62
Reject005=1 : Homoscedasticity      38
Name: Result_test, dtype: int64